In [1]:
from mplsoccer.pitch import Pitch, VerticalPitch
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from highlight_text import HighlightText, ax_text, fig_text
import numpy as np
from matplotlib.colors import LinearSegmentedColormap
from scipy.ndimage import gaussian_filter
import matplotlib.patheffects as path_effects

In [ ]:
data = pd.read_csv("epl_22_23.csv" , low_memory=False)
data['x'] = data['x']*1.2
data['y'] = data['y']*.8
data['endX'] = data['endX']*1.2
data['endY'] = data['endY']*.8

In [ ]:
cmap_lst = ['#131313', '#24a8ff', '#0d4163'] 
cmap=LinearSegmentedColormap.from_list('test',cmap_lst, N=256) 

In [ ]:
def add_carries(game_df):
   min_dribble_length: float = 0.0
   max_dribble_length: float = 100.0
   max_dribble_duration: float = 20.0
   game_df['time_seconds']=game_df['minute']*60+game_df['second']
   next_actions = data.shift(-1)
   same_game=game_df.matchId==next_actions.matchId
   same_team = game_df.teamId == next_actions.teamId
   dx = game_df.endX - next_actions.x
   dy = game_df.endY- next_actions.y
   far_enough=dy** 2 + dy** 2 >= min_dribble_length** 2
   not_too_far = dx ** 2+ dy** 2<= max_dribble_length ** 2
   dt = next_actions.time_seconds - game_df.time_seconds
   same_phase = dt < max_dribble_duration
   same_period = game_df.period == next_actions.period
   dribble_idx = same_team & far_enough & not_too_far & same_phase & same_period&same_game
   dribbles = pd.DataFrame()
   prev = game_df[dribble_idx]
   nex= next_actions[dribble_idx]
   dribbles['matchId' ] = nex.matchId
   dribbles['period' ] = nex.period
   for cols in ['expandedMinute']:
      dribbles[cols] = nex[cols]
   for cols in ['passKey','assist']:
      dribbles[ cols] = [True for _ in range (len (dribbles))]
   dribbles['isTouch'] = [True for _ in range (len (dribbles))]
   morecols = ['playerId', 'team_name','playerName','h_a']
   for cols in morecols:
      dribbles[cols] = nex[cols]
   dribbles['action_id'] = prev.eventId + 0.1
   dribbles['time_seconds' ] = (prev.time_seconds + nex.time_seconds) / 2
   dribbles['teamId'] = nex.teamId
   dribbles['playerId']= nex.playerId
   dribbles['playerName' ]= nex.playerName
   dribbles['x'] = prev.endX
   dribbles['y']= prev.endY
   dribbles['endX'] = nex.x
   dribbles['endY'] = nex.y
   dribbles['type'] = ['Carry' for _ in range(len(dribbles))]
   dribbles['outcomeType'] = ['Successful' for _ in range(len(dribbles))]
   dribbles['qualifiers']=[{} for _ in range(len(dribbles))]
   game_df=pd.concat([game_df,dribbles], ignore_index=True,sort=False)
   game_df=game_df.sort_values(['matchId','period','action_id']).reset_index(drop=True)
   game_df['action_id']=range(len(game_df))
   return game_df

In [ ]:
data = add_carries(data)

In [ ]:
data_andre = data[data['playerId'] == 113069]

In [ ]:
data_andre

In [ ]:
data_dribbles = data_andre[data_andre['type'] == 'TakeOn']

In [ ]:
data_tackles = data_andre[data_andre['type'] == 'Tackle']

In [ ]:
tackles_won = data_tackles[data_tackles['tackleWon'] == True]
tackles_lost = data_tackles[data_tackles['tackleLost'] == True]

In [ ]:
fig ,ax = plt.subplots(figsize=(15.5,12))
fig.set_facecolor('#242526')
ax.patch.set_facecolor('#242526')

pitch = Pitch(pitch_type='statsbomb', orientation='horizontal',
              pitch_color='#1e1e1e', line_color='#FFFFFF', figsize=(15, 12),
              constrained_layout=False, tight_layout=True)

pitch.draw(ax=ax)


#Plotting Tackles Won

pitch.scatter(tackles_won.x, tackles_won.y, s=420,
              marker='X', edgecolors='#24a8ff', c='none', zorder=4, ax=ax , alpha=1 , ls='-' , lw=1.6)

pitch.scatter(tackles_won.x, tackles_won.y, s=420,
              marker='X', edgecolors='none', c='#24a8ff', zorder=4, ax=ax , alpha=0.70)

#Plotting Tackles Lost

pitch.scatter(tackles_lost.x, tackles_lost.y, s=300,
              marker='X', edgecolors='#949494', c='none', zorder=3, ax=ax , alpha=0.4 , ls='-' , lw=1.6)

pitch.scatter(tackles_lost.x, tackles_lost.y, s=300,
              marker='X', edgecolors='none', c='#949494', zorder=3, ax=ax , alpha=0.25)




plt.gca().invert_yaxis()

fig_text(
    0.512, 0.945, "<Gini Wijnaldum>", font='Franklin Gothic Medium' , size=30,
    ha="center", color="#FFFFFF" , fontweight='bold' , highlight_textprops=[{"color": '#FFFFFF'}]
)

fig_text(
    0.518, 0.898,
    "Tackles Map | Liverpool 2018-19 | 2725 Minutes",
    font='Franklin Gothic Medium' , size=24,
    ha="center", color="#FFFFFF" , fontweight='bold'
)




fig_text(
    0.514, 0.851,
    "<28 Tackles Won> | 41 Tackles Lost | 68.3% Accuracy",
    font='Franklin Gothic Medium' , size=24,
    ha="center", color="#FFFFFF" , fontweight='bold' , highlight_textprops=[{"color" : "#24a8ff"}]
)




fig_text(
    0.790, 0.135, "Made by: @pranav_m28", font='Franklin Gothic Medium' , size=18,
    ha="center", color="#FFFFFF" , fontweight='bold'
)



fig_text(
    0.510, 0.150, "Direction of Play", font='Franklin Gothic Medium' , size=18,
    ha="center", color="#FFFFFF" , fontweight='bold'
)

plt.arrow(49.2 , -3 , 20 , 0 , fc='#FFFFFF' , ls='-' , lw=1.9 , head_length=1 , head_width=1)



plt.style.use("dark_background")
mpl.rc('axes',edgecolor='#131313' , linewidth=1.2)
for param in ['figure.facecolor', 'axes.facecolor', 'savefig.facecolor']:
    plt.rcParams[param] = '#1e1e1e' 
for param in ['text.color', 'axes.labelcolor', 'xtick.color', 'ytick.color']:
    plt.rcParams[param] = '0.9'
    

pitch.lines(40 , 0 , 40 , 80 , ls='--' , lw=1.5 , color='#FFFFFF' , ax=ax , zorder=1 , alpha=0.7)

pitch.lines(80 , 0 , 80 , 80 , ls='--' , lw=1.5 , color='#FFFFFF' , ax=ax , zorder=1 , alpha=0.7)   


#plt.savefig("Gini 18-19 Tackling.png" , dpi=300, bbox_inches='tight')

In [ ]:
tackles_won['x'].count()

In [ ]:
tackles_lost['x'].count()

In [ ]:
41/57

In [ ]:
data_dribbles

In [ ]:
dribbles_won = data_dribbles[data_dribbles['dribbleWon'] == True]
dribbles_lost = data_dribbles[data_dribbles['dribbleLost'] == True]

In [ ]:
dribbles_won['x'].count()

In [ ]:
dribbles_lost['x'].count()

In [ ]:
fig ,ax = plt.subplots(figsize=(15.5,12))
fig.set_facecolor('#242526')
ax.patch.set_facecolor('#242526')

pitch = Pitch(pitch_type='statsbomb', orientation='horizontal',
              pitch_color='#1e1e1e', line_color='#FFFFFF', figsize=(15, 12),
              constrained_layout=False, tight_layout=True)

pitch.draw(ax=ax)


#Plotting TakeOns Won

pitch.scatter(dribbles_won.x, dribbles_won.y, s=420,
              marker='X', edgecolors='#24a8ff', c='none', zorder=4, ax=ax , alpha=1 , ls='-' , lw=1.6)

pitch.scatter(dribbles_won.x, dribbles_won.y, s=420,
              marker='X', edgecolors='none', c='#24a8ff', zorder=4, ax=ax , alpha=0.70)

#Plotting TakeOns Lost

pitch.scatter(dribbles_lost.x, dribbles_lost.y, s=300,
              marker='X', edgecolors='#949494', c='none', zorder=3, ax=ax , alpha=0.4 , ls='-' , lw=1.6)

pitch.scatter(dribbles_lost.x, dribbles_lost.y, s=300,
              marker='X', edgecolors='none', c='#949494', zorder=3, ax=ax , alpha=0.25)




plt.gca().invert_yaxis()

fig_text(
    0.512, 0.945, "<Gini Wijnaldum>", font='Franklin Gothic Medium' , size=30,
    ha="center", color="#FFFFFF" , fontweight='bold' , highlight_textprops=[{"color": '#FFFFFF'}]
)

fig_text(
    0.518, 0.898,
    "Dribbling Map Liverpool 2018-19 | 2725 Minutes",
    font='Franklin Gothic Medium' , size=24,
    ha="center", color="#FFFFFF" , fontweight='bold'
)




fig_text(
    0.514, 0.851,
    "<29 Dribbles Won> | 20 Dribbles Lost | 59.2% Accuracy",
    font='Franklin Gothic Medium' , size=24,
    ha="center", color="#FFFFFF" , fontweight='bold' , highlight_textprops=[{"color" : "#24a8ff"}]
)




fig_text(
    0.790, 0.135, "Made by: @pranav_m28", font='Franklin Gothic Medium' , size=18,
    ha="center", color="#FFFFFF" , fontweight='bold'
)



fig_text(
    0.510, 0.150, "Direction of Play", font='Franklin Gothic Medium' , size=18,
    ha="center", color="#FFFFFF" , fontweight='bold'
)

plt.arrow(49.2 , -3 , 20 , 0 , fc='#FFFFFF' , ls='-' , lw=1.9 , head_length=1 , head_width=1)



plt.style.use("dark_background")
mpl.rc('axes',edgecolor='#131313' , linewidth=1.2)
for param in ['figure.facecolor', 'axes.facecolor', 'savefig.facecolor']:
    plt.rcParams[param] = '#1e1e1e' 
for param in ['text.color', 'axes.labelcolor', 'xtick.color', 'ytick.color']:
    plt.rcParams[param] = '0.9'
    

pitch.lines(40 , 0 , 40 , 80 , ls='--' , lw=1.5 , color='#FFFFFF' , ax=ax , zorder=1 , alpha=0.7)

pitch.lines(80 , 0 , 80 , 80 , ls='--' , lw=1.5 , color='#FFFFFF' , ax=ax , zorder=1 , alpha=0.7)   


#plt.savefig("Gini 18-19 Dribbling.png" , dpi=300, bbox_inches='tight')

In [ ]:
dribbles_won['x'].count()

In [ ]:
recov = data_andre[data_andre['ballRecovery'] == True]

In [ ]:
recov

In [ ]:
fig ,ax = plt.subplots(figsize=(15.5,12))
fig.set_facecolor('#1e1e1e')
ax.patch.set_facecolor('#1e1e1e')

pitch = Pitch(pitch_type='statsbomb', orientation='horizontal',
              pitch_color='#1e1e1e', line_color='#FFFFFF', figsize=(15, 12),
              constrained_layout=False, tight_layout=True , line_zorder=2)

pitch.draw(ax=ax)

#bin_statistic = pitch.bin_statistic(recov.x, recov.y, statistic='count', bins=(25, 25))
#bin_statistic['statistic'] = gaussian_filter(bin_statistic['statistic'], 1)
#pcm = pitch.heatmap(bin_statistic, ax=ax, cmap=cmap, edgecolors='#000000' , alpha=0.88)

pitch.scatter(recov.x, recov.y, s=420,
              marker='X', edgecolors='#24a8ff', c='none', zorder=4, ax=ax , alpha=1 , ls='-' , lw=1.6)

pitch.scatter(recov.x, recov.y, s=420,
              marker='X', edgecolors='none', c='#24a8ff', zorder=4, ax=ax , alpha=0.70)




plt.gca().invert_yaxis()


fig_text(
    0.512, 0.945, "<Alexis Mac Allister>", font='Franklin Gothic Medium' ,size=30,
    ha="center", color="#FFFFFF" , fontweight='bold' , highlight_textprops=[{"color": '#FFFFFF'}]
)

fig_text(
    0.518, 0.898,
    "Recovery Map | Brighton 2022-23",
    font='Franklin Gothic Medium' ,size=24,
    ha="center", color="#FFFFFF" , fontweight='bold'
)

fig_text(
    0.514, 0.851,
    "Recoveries = 213 (6.64 per 90)",
    font='Franklin Gothic Medium' ,size=20,
    ha="center", color="#FFFFFF" , fontweight='bold'
)


fig_text(
    0.514, 0.851,
    "<>",
    size=13,
    ha="center", color="#FFFFFF" , fontweight='bold' , highlight_textprops=[{"color": '#FFFFFF'}]
)




fig_text(
    0.790, 0.135, "Made by: @pranav_m28", font='Franklin Gothic Medium' ,size=18,
    ha="center", color="#FFFFFF" , fontweight='bold'
)



fig_text(
    0.510, 0.150, "Direction of Play", font='Franklin Gothic Medium' ,size=18,
    ha="center", color="#FFFFFF" , fontweight='bold'
)

plt.arrow(49.2 , -3 , 20 , 0 , fc='#FFFFFF' , ls='-' , lw=1.9 , head_length=1 , head_width=1)



plt.style.use("dark_background")
mpl.rc('axes',edgecolor='#131313' , linewidth=1.2)
for param in ['figure.facecolor', 'axes.facecolor', 'savefig.facecolor']:
    plt.rcParams[param] = '#1e1e1e' 
for param in ['text.color', 'axes.labelcolor', 'xtick.color', 'ytick.color']:
    plt.rcParams[param] = '0.9'
    

pitch.lines(40 , 0 , 40 , 80 , ls='--' , lw=1.5 , color='#FFFFFF' , ax=ax , zorder=1 , alpha=0.7)

pitch.lines(80 , 0 , 80 , 80 , ls='--' , lw=1.5 , color='#FFFFFF' , ax=ax , zorder=1 , alpha=0.7)  

#plt.savefig('AMA Recoveries.png' , dpi=300 , bbox_inches='tight')

In [ ]:
recov['x'].count()

In [ ]:
213/32.1

In [ ]:
73/105

In [ ]:
##Passing Vizzes
data_passes = data_andre[data_andre['type'] == 'Pass']

In [ ]:
data_passes

In [ ]:
data_succ = data_passes[data_passes['outcomeType'] == 'Successful']
data_unsucc = data_passes[data_passes['outcomeType'] == 'Unsuccessful']

In [ ]:
data_succ_prog = data_succ.reset_index()
data_unsucc_prog = data_unsucc.reset_index()
data_succ_prog['beginning'] = np.sqrt(np.square(120-data_succ_prog['x']) + np.square(40 - data_succ_prog['y']))
data_succ_prog['end'] = np.sqrt(np.square(120 - data_succ_prog['endX']) + np.square(40 - data_succ_prog['endY']))

data_succ_prog['progressive'] = [(data_succ_prog['end'][x]) / (data_succ_prog['beginning'][x]) < .75 for x in range(len(data_succ_prog.beginning))]
data_succ_prog = data_succ_prog[data_succ_prog['progressive'] == True]

In [ ]:
data_carries = data_andre[data_andre['type'] == 'Carry']

data_carries_succ = data_carries[data_carries['outcomeType'] == 'Successful']
data_carries_unsucc = data_carries[data_carries['outcomeType'] == 'Unsuccessful']

data_carries_succ_prog = data_carries_succ.reset_index()
#data_unsucc_prog = data_unsucc.reset_index()

data_carries_succ_prog['beginning'] = np.sqrt(np.square(120-data_carries_succ_prog['x']) + np.square(40 - data_carries_succ_prog['y']))
data_carries_succ_prog['end'] = np.sqrt(np.square(120 - data_carries_succ_prog['endX']) + np.square(40 - data_carries_succ_prog['endY']))

data_carries_succ_prog['progressive'] = [(data_carries_succ_prog['end'][x]) / (data_carries_succ_prog['beginning'][x]) < .75 for x in range(len(data_carries_succ_prog.beginning))]
data_carries_succ_prog = data_carries_succ_prog[data_carries_succ_prog['progressive'] == True]

In [ ]:
data_carries_succ_prog['x'].count()

In [ ]:
data_succ_prog = data_succ_prog[data_succ_prog['passCorner'] == False]

In [ ]:
fig ,ax = plt.subplots(figsize=(15.5,10))
fig.set_facecolor('#242526')
ax.patch.set_facecolor('#242526')

pitch = Pitch(pitch_type='statsbomb', orientation='horizontal',
              pitch_color='#1e1e1e', line_color='#FFFFFF', figsize=(15, 10),
              constrained_layout=False, tight_layout=True)

pitch.draw(ax=ax)

#Plotting Progressive Passes

lc1 = pitch.lines(data_succ_prog.x, data_succ_prog.y,
                  data_succ_prog.endX, data_succ_prog.endY,
                  lw=4, transparent=True, comet=True, label='completed passes',
                  color='#24a8ff', ax=ax)

pitch.scatter(data_succ_prog.endX, data_succ_prog.endY, s=70,
              marker='o', edgecolors='none', c='#24a8ff', zorder=2,
              label='goal', ax=ax , alpha=1)


#Plotting Progressive Carries

lc1 = pitch.lines(data_carries_succ_prog.x, data_carries_succ_prog.y,
                  data_carries_succ_prog.endX, data_carries_succ_prog.endY,ls='dashed',
                  lw=3, transparent=False, comet=False, label='completed passes',
                  color='#FF5959', ax=ax)

pitch.scatter(data_carries_succ_prog.endX, data_carries_succ_prog.endY, s=70,
              marker='o', edgecolors='none', c='#FF5959', zorder=2,
              label='goal', ax=ax , alpha=1)

plt.gca().invert_yaxis()


fig_text(
    0.516, 0.99, "<Trent Alexander-Arnold>", font='Franklin Gothic Medium' ,size=30,
    ha="center", color="#FFFFFF" , fontweight='bold' , highlight_textprops=[{"color": '#FFFFFF'}]
)

fig_text(
    0.518, 0.941,
    "Progressive Actions Map | Liverpool 2022-23",
    font='Franklin Gothic Medium' ,size=24,
    ha="center", color="#FFFFFF" , fontweight='bold'
)

fig_text(
    0.518, 0.892,
    "Progressive Actions = 386 (11.88 per 90)",
    font='Franklin Gothic Medium' ,size=20,
    ha="center", color="#FFFFFF" , fontweight='bold'
)

fig_text(
    0.770, 0.105, "Made by: @pranav_m28\nData: Opta", font='Franklin Gothic Medium' ,size=18,
    ha="center", color="#FFFFFF" , fontweight='bold'
)

fig_text(
    0.250, 0.105, "<Prog. Passes - 365>\n<Prog. Carries - 21>", font='Franklin Gothic Medium' ,size=18,
    ha="center", color="#FFFFFF" , fontweight='bold' , highlight_textprops=[{"color": '#24a8ff'} , {"color": '#FF5959'}]
)

plt.style.use("dark_background")
mpl.rc('axes',edgecolor='#131313' , linewidth=1.2)
for param in ['figure.facecolor', 'axes.facecolor', 'savefig.facecolor']:
    plt.rcParams[param] = '#1e1e1e' 
for param in ['text.color', 'axes.labelcolor', 'xtick.color', 'ytick.color']:
    plt.rcParams[param] = '0.9'


fig_text(
    0.510, 0.115, "Direction of Play", font='Franklin Gothic Medium' ,size=18,
    ha="center", color="#FFFFFF" , fontweight='bold'
)

plt.arrow(49.2 , -3 , 20 , 0 , fc='#FFFFFF' , ls='-' , lw=1.9 , head_length=1 , head_width=1)

#plt.savefig('Trent Progressive Actions 22-23.png' , dpi=300 , bbox_inches='tight')


In [ ]:
data_succ_prog['x'].count()

In [ ]:
386/32.5

In [ ]:
cmap_lst = ['#131313', '#24a8ff', '#0d4163'] 
cmap=LinearSegmentedColormap.from_list('test',cmap_lst, N=256) 

In [ ]:
fig ,ax = plt.subplots(figsize=(15.5,12))
fig.set_facecolor('#1e1e1e')
ax.patch.set_facecolor('#1e1e1e')

pitch = Pitch(pitch_type='statsbomb', orientation='horizontal',
              pitch_color='#1e1e1e', line_color='#FFFFFF', figsize=(15, 12),
              constrained_layout=False, tight_layout=True , line_zorder=2)

pitch.draw(ax=ax)

bin_statistic = pitch.bin_statistic(data_succ_prog.endX, data_succ_prog.endY, statistic='count', bins=(25, 25))
bin_statistic['statistic'] = gaussian_filter(bin_statistic['statistic'], 1)
pcm = pitch.heatmap(bin_statistic, ax=ax, cmap=cmap, edgecolors='#000000' , alpha=0.88)

plt.gca().invert_yaxis()


fig_text(
    0.516, 0.99, "<Trent Alexander-Arnold>", font='Franklin Gothic Medium' ,size=30,
    ha="center", color="#FFFFFF" , fontweight='bold' , highlight_textprops=[{"color": '#24a8ff'}]
)

fig_text(
    0.518, 0.941,
    "Progressive Passes End Locations | Liverpool 2022-23",
    font='Franklin Gothic Medium' ,size=24,
    ha="center", color="#FFFFFF" , fontweight='bold'
)

fig_text(
    0.518, 0.892,
    "Progressive Passes = 263 (12.82 per 90)",
    font='Franklin Gothic Medium' ,size=20,
    ha="center", color="#FFFFFF" , fontweight='bold'
)


fig_text(
    0.514, 0.851,
    "<>",
    size=13,
    ha="center", color="#FFFFFF" , fontweight='bold' , highlight_textprops=[{"color": '#FFFFFF'}]
)




fig_text(
    0.790, 0.135, "Made by: @pranav_m28", font='Franklin Gothic Medium' ,size=18,
    ha="center", color="#FFFFFF" , fontweight='bold'
)



fig_text(
    0.510, 0.150, "Direction of Play", font='Franklin Gothic Medium' ,size=18,
    ha="center", color="#FFFFFF" , fontweight='bold'
)

plt.arrow(49.2 , -3 , 20 , 0 , fc='#FFFFFF' , ls='-' , lw=1.9 , head_length=1 , head_width=1)



plt.style.use("dark_background")
mpl.rc('axes',edgecolor='#131313' , linewidth=1.2)
for param in ['figure.facecolor', 'axes.facecolor', 'savefig.facecolor']:
    plt.rcParams[param] = '#1e1e1e' 
for param in ['text.color', 'axes.labelcolor', 'xtick.color', 'ytick.color']:
    plt.rcParams[param] = '0.9'
    

pitch.lines(40 , 0 , 40 , 80 , ls='--' , lw=1.5 , color='#FFFFFF' , ax=ax , zorder=1 , alpha=0.7)

pitch.lines(80 , 0 , 80 , 80 , ls='--' , lw=1.5 , color='#FFFFFF' , ax=ax , zorder=1 , alpha=0.7)  

#plt.savefig('Andre Progressive Passes End Locations.png' , dpi=300 , bbox_inches='tight')



In [ ]:
recov_mid = recov[(recov['x'] <= 80) & (recov['x'] >= 40)]

In [ ]:
recov_mid['x'].count()

In [ ]:
recov_def = recov[recov['x'] <= 40]

In [ ]:
recov_def['x'].count()

In [ ]:
data_andre['x_n'] = data_andre['x'].shift(-1)
data_andre['y_n'] = data_andre['y'].shift(-1)
data_andre['endX_n'] = data_andre['endX'].shift(-1)
data_andre['endY_n'] = data_andre['endY'].shift(-1)
data_andre['type_n'] = data_andre['type'].shift(-1)
data_andre['outcomeType_n'] = data_andre['outcomeType'].shift(-1)
recov_2 = data_andre[data_andre['type'] == 'BallRecovery']
succ_pass_afrecov = recov_2[recov_2['outcomeType_n'] == 'Successful']
succ_pass_afrecov_prog = succ_pass_afrecov.reset_index()
#data_unsucc_prog = data_unsucc.reset_index()
succ_pass_afrecov_prog['beginning'] = np.sqrt(np.square(120-succ_pass_afrecov_prog['x_n']) + np.square(40 - succ_pass_afrecov_prog['y_n']))
succ_pass_afrecov_prog['end'] = np.sqrt(np.square(120 - succ_pass_afrecov_prog['endX_n']) + np.square(40 - succ_pass_afrecov_prog['endY_n']))

succ_pass_afrecov_prog['progressive'] = [(succ_pass_afrecov_prog['end'][x]) / (succ_pass_afrecov_prog['beginning'][x]) < .75 for x in range(len(succ_pass_afrecov_prog.beginning))]
succ_pass_afrecov_prog = succ_pass_afrecov_prog[succ_pass_afrecov_prog['progressive'] == True]

#Unsucc Passes

unsucc_pass_afrecov = recov_2[recov_2['outcomeType_n'] == 'Unsuccessful']
unsucc_pass_afrecov_prog = unsucc_pass_afrecov.reset_index()
#data_unsucc_prog = data_unsucc.reset_index()
unsucc_pass_afrecov_prog['beginning'] = np.sqrt(np.square(120-unsucc_pass_afrecov_prog['x_n']) + np.square(40 - unsucc_pass_afrecov_prog['y_n']))
unsucc_pass_afrecov_prog['end'] = np.sqrt(np.square(120 - unsucc_pass_afrecov_prog['endX_n']) + np.square(40 - unsucc_pass_afrecov_prog['endY_n']))

unsucc_pass_afrecov_prog['progressive'] = [(unsucc_pass_afrecov_prog['end'][x]) / (unsucc_pass_afrecov_prog['beginning'][x]) < .75 for x in range(len(unsucc_pass_afrecov_prog.beginning))]
unsucc_pass_afrecov_prog = unsucc_pass_afrecov_prog[unsucc_pass_afrecov_prog['progressive'] == True]




In [ ]:
unsucc_pass_afrecov['endX_n'].count()

In [ ]:
fig ,ax = plt.subplots(figsize=(15.5,10))
fig.set_facecolor('#242526')
ax.patch.set_facecolor('#242526')

pitch = Pitch(pitch_type='statsbomb', orientation='horizontal',
              pitch_color='#1e1e1e', line_color='#FFFFFF', figsize=(15, 10),
              constrained_layout=False, tight_layout=True)

pitch.draw(ax=ax)

#Plotting Passes post Recovery

lc1 = pitch.lines(succ_pass_afrecov.x_n, succ_pass_afrecov.y_n,
                  succ_pass_afrecov.endX_n, succ_pass_afrecov.endY_n,
                  lw=4, transparent=True, comet=True, label='completed passes',
                  color='#24a8ff', ax=ax , ZORDER=3)

pitch.scatter(succ_pass_afrecov.endX_n, succ_pass_afrecov.endY_n, s=70,
              marker='o', edgecolors='none', c='#24a8ff', zorder=3,
              label='goal', ax=ax , alpha=1)

#Plotting Prog Passes post Recovery

lc1 = pitch.lines(succ_pass_afrecov_prog.x_n, succ_pass_afrecov_prog.y_n,
                  succ_pass_afrecov_prog.endX_n, succ_pass_afrecov_prog.endY_n,
                  lw=4, transparent=True, comet=True, label='completed passes',
                  color='#03fc24', ax=ax, ZORDER=4)

pitch.scatter(succ_pass_afrecov_prog.endX_n, succ_pass_afrecov_prog.endY_n, s=70,
              marker='o', edgecolors='none', c='#03fc24', zorder=4,
              label='goal', ax=ax , alpha=1)


#Plotting Unsuccessful Passes post Recovery

lc1 = pitch.lines(unsucc_pass_afrecov.x_n, unsucc_pass_afrecov.y_n,
                  unsucc_pass_afrecov.endX_n, unsucc_pass_afrecov.endY_n,
                  lw=4, transparent=True, comet=True, label='completed passes',
                  color='#FF5959', ax=ax , alpha=0.5)

pitch.scatter(unsucc_pass_afrecov.endX_n, unsucc_pass_afrecov.endY_n, s=70,
              marker='o', edgecolors='none', c='#FF5959', zorder=1,
              label='goal', ax=ax , alpha=0.8)



plt.gca().invert_yaxis()


fig_text(
    0.516, 0.99, "<Martin Odegaard>", font='Franklin Gothic Medium' ,size=30,
    ha="center", color="#FFFFFF" , fontweight='bold' , highlight_textprops=[{"color": '#FFFFFF'}]
)

fig_text(
    0.518, 0.941,
    "Successful Passes after Ball Recovery | ARS 2022-23 | 34.7 90s Played",
    font='Franklin Gothic Medium' ,size=24,
    ha="center", color="#FFFFFF" , fontweight='bold'
)

fig_text(
    0.518, 0.892,
    "80% of the Balls that Odegaard Recovers end up in a successful pass by him ",
    font='Franklin Gothic Medium' ,size=20,
    ha="center", color="#FFFFFF" , fontweight='bold'
)

fig_text(
    0.770, 0.105, "Made by: @pranav_m28\nData: Opta\nPL 2022-23", font='Franklin Gothic Medium' ,size=18,
    ha="center", color="#FFFFFF" , fontweight='bold'
)

fig_text(
    0.265, 0.105, "<Successful Passes = 142 (4.10)>\n<Progressive Passes = 15 (0.43)>\n<Unsuccessful Passes = 36 (1.04)>",
    font='Franklin Gothic Medium' ,size=16,
    ha="center", color="#FFFFFF" , fontweight='bold' , highlight_textprops=[{"color": '#24a8ff'} , {"color": '#03fc24'}
                                                                           , {"color" : "#FF5959"}]
)

plt.style.use("dark_background")
mpl.rc('axes',edgecolor='#131313' , linewidth=1.2)
for param in ['figure.facecolor', 'axes.facecolor', 'savefig.facecolor']:
    plt.rcParams[param] = '#1e1e1e' 
for param in ['text.color', 'axes.labelcolor', 'xtick.color', 'ytick.color']:
    plt.rcParams[param] = '0.9'


fig_text(
    0.510, 0.115, "Direction of Play", font='Franklin Gothic Medium' ,size=18,
    ha="center", color="#FFFFFF" , fontweight='bold'
)

plt.arrow(49.2 , -3 , 20 , 0 , fc='#FFFFFF' , ls='-' , lw=1.9 , head_length=1 , head_width=1)

#plt.savefig('Odegaard 22-23 post-Recovery pass map.png' , dpi=300 , bbox_inches='tight')

In [ ]:
succ_pass_afrecov_prog['x'].count()

In [ ]:
succ_pass_afrecov['x'].count()

In [ ]:
unsucc_pass_afrecov['x'].count()

In [ ]:
succ_pass_afrecov['x'].count()/(succ_pass_afrecov['x'].count() + unsucc_pass_afrecov['x'].count())

In [ ]:
142/34.7

In [ ]:
18/301

In [ ]:
18/32.3

In [ ]:
39/32.3

data